<h1>Testing out Jupyter and yahoo finance API with pandas</h1>

Using yahoo finance as a data source to start via yfinance.  Since the yahoo finance API is no longer being supported, yfinance offers an alternative based on scraping the yahoo finance website.  

To run a symbol through all code below, simply change variable <b>ticker</b> below and run all code. 


In [2]:
## this code is an example of how yfinance can be used with pandas.  it overrides the pandasdatareader
## functionality that makes use of the deprecated yahoo api
## ideally i would like to just use yfinance without the override to reduce deps and complexity
## but will see how it goes.

import pandas as pd
import pandas_datareader.data as pdr
import yfinance as yf

yf.pdr_override()
# global test parameters for workflow
ticker = 'T.TO'
start_date = '2010-01-01'
end_date = '2020-01-20'

yahoo_price_check = pdr.DataReader(ticker,  start=start_date, end=end_date, data_source='yahoo')
yahoo_price_check.reset_index(inplace=True,drop=False)
print(yahoo_price_check)


[*********************100%***********************]  1 of 1 completed
           Date       Open       High        Low      Close  Adj Close  \
0    2010-01-04   8.542500   8.592500   8.490000   8.547500   5.459784   
1    2010-01-05   8.575000   8.640000   8.487500   8.540000   5.454991   
2    2010-01-06   8.540000   8.622500   8.447500   8.547500   5.459784   
3    2010-01-07   8.547500   8.547500   8.500000   8.515000   5.439023   
4    2010-01-08   8.525000   8.587500   8.522500   8.580000   5.480542   
...         ...        ...        ...        ...        ...        ...   
2515 2020-01-13  25.155001  25.205000  25.030001  25.180000  24.585207   
2516 2020-01-14  25.225000  25.225000  25.049999  25.139999  24.546148   
2517 2020-01-15  25.125000  25.254999  25.065001  25.215000  24.619379   
2518 2020-01-16  25.225000  25.455000  25.139999  25.424999  24.824417   
2519 2020-01-17  25.434999  25.655001  25.365000  25.650000  25.044102   

       Volume  
0     1380400  
1     2196

    Alright so that is working ok.  Now lets get some dividend.  May need to try out adj close to account for splits
    
    so the date is used as index but cannot be referenced for joining later.  I reset the index to generic integer based index.  Doing thing on the price dataframe gives a regular Date column but for some reason, the same conversion leaves date labelled 'index' for yahoo-actions.  Quick relabel after reindexing should make this joinable. 

In [3]:
yf.pdr_override()

yahoo_div_check = pdr.DataReader(ticker, start=start_date, end=end_date, data_source='yahoo-actions')
yahoo_div_check.reset_index(inplace=True,drop=False)
yahoo_div_check = yahoo_div_check.rename(columns={'index':'Date'})
print(yahoo_div_check)

[*********************100%***********************]  1 of 1 completed
           Date       Open       High        Low      Close  Adj Close  \
0    2010-01-04   8.542500   8.592500   8.490000   8.547500   5.459784   
1    2010-01-05   8.575000   8.640000   8.487500   8.540000   5.454991   
2    2010-01-06   8.540000   8.622500   8.447500   8.547500   5.459784   
3    2010-01-07   8.547500   8.547500   8.500000   8.515000   5.439023   
4    2010-01-08   8.525000   8.587500   8.522500   8.580000   5.480542   
...         ...        ...        ...        ...        ...        ...   
2515 2020-01-13  25.155001  25.205000  25.030001  25.180000  24.585207   
2516 2020-01-14  25.225000  25.225000  25.049999  25.139999  24.546148   
2517 2020-01-15  25.125000  25.254999  25.065001  25.215000  24.619379   
2518 2020-01-16  25.225000  25.455000  25.139999  25.424999  24.824417   
2519 2020-01-17  25.434999  25.655001  25.365000  25.650000  25.044102   

       Volume  
0     1380400  
1     2196

OK so the yahoo-actions override is not working ... no worries.  yfinance refactor may be the better option.  
Test sections below.

In [ ]:
import yfinance as yf
print(yf.__version__)

In [11]:
# what does financials, balance sheet, and cashflow return
# alright so there was a bug in the yfiannce library (v. 0.1.54)
# I found a fix for now until a patch is pushed to master on this.
# waiting on this PR for permanent fix : https://github.com/ranaroussi/yfinance/pull/246
# implemented fix from issue here: https://github.com/ranaroussi/yfinance/issues/234

import yfinance as yf
ticker = yf.Ticker("BCE.TO")

fins = ticker.balance_sheet
#fins = ticker.earnings
#fins = yf.__file__

print(fins)

                                    2019-12-31    2018-12-31    2017-12-31  \
Intangible Assets                 1.335200e+10  1.320500e+10  1.325800e+10   
Capital Surplus                   1.178000e+09  1.170000e+09  1.162000e+09   
Total Liab                        3.873800e+10  3.641100e+10  3.517700e+10   
Total Stockholder Equity          1.707000e+10  1.635900e+10  1.629800e+10   
Minority Interest                 3.340000e+08  3.260000e+08  3.230000e+08   
Deferred Long Term Liab           3.680000e+08  3.370000e+08  2.860000e+08   
Other Current Liab                2.340000e+09  2.272000e+09  2.199000e+09   
Total Assets                      6.014600e+10  5.710000e+10  5.580200e+10   
Common Stock                      2.036300e+10  2.003600e+10  2.009100e+10   
Other Current Assets              6.050000e+08  6.990000e+08  4.720000e+08   
Retained Earnings                -4.632000e+09 -4.937000e+09 -4.938000e+09   
Other Liab                        6.546000e+09  6.222000e+09  6.

In [ ]:
## heres the list of all the various functions to pull back different data fields for each ticker
## explore these individually to see how i want to piece them together

import yfinance as yf

msft = yf.Ticker("MSFT")

# get stock info
print(msft.info)

# get historical market data
hist = msft.history(period="max")

# show actions (dividends, splits)
msft.actions

# show dividends
msft.dividends

# show splits
msft.splits

# show financials
msft.financials
msft.quarterly_financials

# show major holders
msft.major_holders

# show institutional holders
msft.institutional_holders

# show balance heet
msft.balance_sheet
msft.quarterly_balance_sheet

# show cashflow
msft.cashflow
msft.quarterly_cashflow

# show earnings
msft.earnings
msft.quarterly_earnings

# show sustainability
msft.sustainability

# show analysts recommendations
msft.recommendations

# show next event (earnings, etc)
msft.calendar

# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
#msft.isin

# show options expirations
#msft.options

# get option chain for specific expiration
#opt = msft.option_chain('YYYY-MM-DD')
# data available via: opt.calls, opt.puts

So looking at data types returned by yfinance calls:

The Ticker object returned contains a ton of information in various formats (dicts, dataframes, etc) and provides all the access methods to pull back more.  

history function returns Date, open, high, low, volume, divs, and splits.  I will keep all of the fields as they may come in useful in the future.  Returned as a Pandas dataframe. 

In [6]:
import yfinance as yf
import pandas as pd

msft = yf.Ticker("T.TO")

# get stock info : returns a dict with all general inforation about the company.  
msft.info

hist = msft.history(period="max", auto_adjust=False)
print(hist)

             Open   High    Low  Close  Adj Close   Volume  Dividends  \
Date                                                                    
1995-01-12   5.94   5.94   5.84   5.91       2.14   394400        0.0   
1995-01-13   5.94   5.94   5.78   5.81       2.10    76000        0.0   
1995-01-16   5.81   5.84   5.78   5.81       2.10    70400        0.0   
1995-01-17   5.78   5.78   5.69   5.75       2.08   112400        0.0   
1995-01-18   5.69   5.72   5.69   5.69       2.06    60800        0.0   
...           ...    ...    ...    ...        ...      ...        ...   
2020-08-20  24.21  24.33  24.08  24.15      24.15  1892400        0.0   
2020-08-21  24.11  24.21  23.96  24.21      24.21  1205100        0.0   
2020-08-24  24.21  24.54  24.18  24.43      24.43  3129500        0.0   
2020-08-25  24.40  24.50  24.25  24.49      24.49  1160300        0.0   
2020-08-26  24.48  24.50  24.21  24.29      24.29   586697        0.0   

            Stock Splits  
Date                   

<b> Playing with Graphing Data from Yahoo Finance </b>

In [5]:
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.scatter(yahoo_div_check['Date'], yahoo_div_check['value'], color = 'red')
plt.title('Test Yahoo Dividends over time')
plt.xlabel('Date')
plt.ylabel('Dividend Amount')
plt.show()

KeyError: 'value'

In [6]:
#yahoo_div_check = pdr.DataReader(ticker, start='2004-01-01', end='2019-12-30', data_source='yahoo-actions')
print(yahoo_div_check)

           Date       Open       High        Low      Close  Adj Close  \
0    2010-01-04   3.470000   3.480000   3.376670   3.386670   2.692099   
1    2010-01-05   3.401670   3.415000   3.333330   3.370000   2.678848   
2    2010-01-06   3.368330   3.381670   3.343330   3.358330   2.669570   
3    2010-01-07   3.340000   3.441670   3.335000   3.415000   2.714618   
4    2010-01-08   3.405000   3.415000   3.356670   3.370000   2.678848   
...         ...        ...        ...        ...        ...        ...   
2515 2020-01-13  43.299999  43.349998  42.750000  43.049999  42.892498   
2516 2020-01-14  42.959999  43.860001  42.799999  43.790001  43.629795   
2517 2020-01-15  43.759998  44.049999  43.119999  43.959999  43.799168   
2518 2020-01-16  44.049999  44.299999  43.790001  44.250000  44.088108   
2519 2020-01-17  44.290001  44.590000  43.980000  44.430000  44.267448   

       Volume  
0      800418  
1      765234  
2     1322112  
3     2813490  
4     2079372  
...       ...  

Show all non-DIVIDEND actions. 

In [7]:
yahoo_div_check = pdr.DataReader(ticker, start=start_date, end=end_date, data_source='yahoo-actions')
yahoo_div_check = yahoo_div_check[yahoo_div_check.action != "DIVIDEND"]
yahoo_div_check.reset_index(inplace=True,drop=False)
yahoo_div_check = yahoo_div_check.rename(columns={'index':'Date'})
print(yahoo_div_check)

[*********************100%***********************]  1 of 1 completed


AttributeError: 'DataFrame' object has no attribute 'action'

So the outlier was a stock split not a dividend.  Should remove those.  Also whats happening pre-2002?

In [ ]:
yahoo_action_check = pdr.DataReader(ticker, start=start_date, end=end_date, data_source='yahoo-actions')
yahoo_div_check = yahoo_action_check[yahoo_action_check.action == "DIVIDEND"]
yahoo_split_check = yahoo_action_check[yahoo_action_check.action == "SPLIT"]
yahoo_other_check = yahoo_action_check[yahoo_action_check.action != "DIVIDEND"]

yahoo_div_check.reset_index(inplace=True,drop=False)
yahoo_div_check = yahoo_div_check.rename(columns={'index':'Date'})

yahoo_split_check.reset_index(inplace=True,drop=False)
yahoo_split_check = yahoo_split_check.rename(columns={'index':'Date'})

yahoo_other_check.reset_index(inplace=True,drop=False)
yahoo_other_check = yahoo_other_check.rename(columns={'index':'Date'})

print(yahoo_div_check)
print(yahoo_split_check)
print(yahoo_other_check)

<b>so got rid of splits ... only dividends ... now graph to check<b>

In [ ]:
plt.scatter(yahoo_div_check['Date'], yahoo_div_check['value'], color = 'red')
plt.title('Test Yahoo Dividends over time')
plt.xlabel('Date')
plt.ylabel('Dividend Amount')
plt.show()

This graph makes it easy to see whether a stock has a regular dividend payout and periodic increases.  

Drops in dividend payout (see ATD-B.TO) could be caused by stock splits.   Need to check for splits and adjust.

Now lets create a data frame with all the dividend yield values based on div and closing price by day.

In [ ]:
div_yield_frame = yahoo_price_check.copy()
#div_yield_frame = yahoo_price_check.loc[:,'Close']


div_yield_frame = div_yield_frame.drop('High', axis=1)
div_yield_frame = div_yield_frame.drop('Low', axis=1)
div_yield_frame = div_yield_frame.drop('Open', axis=1)
div_yield_frame = div_yield_frame.drop('Volume', axis=1)
div_yield_frame = div_yield_frame.drop('Adj Close', axis=1)
#div_yield_frame = div_yield_frame.drop('index', axis=1)



print(div_yield_frame)

So the merge does a join on date but only keeps results that match.  Need to populate all price closes rows with a dividend yield.  Try a different function.  Something that gives me an outer join (or right join)

currently fixed: auto filled non-div days with NaN.  

Next need to populate all the NaN spots with previous dividend value by date and calculate yield based on price each day. 

Alright now add a column for the yield for each day based on calculation . Dividend/price * 100

In [ ]:
#print(div_yield_frame)
#print(yahoo_div_check)
new_div_yield = pd.merge(div_yield_frame, yahoo_div_check, on="Date", how="left")
new_div_yield = new_div_yield.drop('action', axis=1)

# front fill and calculate yield for each day
# not sure if I need the backfill call to complete the dataframe or not
new_div_yield.fillna(method='ffill', inplace=True)
new_div_yield.fillna(method='bfill', inplace=True)
print(new_div_yield)
new_div_yield['div_yield'] = (4*new_div_yield['value'])/new_div_yield['Close'] *100
print(new_div_yield)

# plot dividend yield over time
plt.plot(new_div_yield['Date'], new_div_yield['div_yield'], color = 'red')
plt.title('Test Yahoo Dividends over time')
plt.xlabel('Date')
plt.ylabel('Dividend Yield')
plt.show()

# plot price over time 
plt.plot(new_div_yield['Date'], new_div_yield['Close'], color = 'green')
plt.title('Test Yahoo Dividends over time')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

 ok great ... but the assumption here is that dividends are paid quarterly.  Need to calculate dividend frequency.  Maybe there is a way to get it.  if not, calculate it.  
 
<b>To do:</b> find a way to automatically calculate dividend payout frequency.  

   <i>Initial idea:</i>  for each dividend payment calculate delta between last and current div date and add to dividend yield DF.  This becomes a multiplier used to calculate div yield.  This should account for varying div payout frequencies across stocks <b>(test cases: T.TO, SJR-B.TO, XQQ)</b> as well as changes in payout frequency for individual stocks <b>(test case: Corus CJR-B)</b> 

hmm... so the closing price is adjusted for splits but the dividends are not.  Will need to account for dividend values that change as a result of splits.  To see this, change ticker to <b>(test case: "ATD-B.TO")</b>. 

<b>To do:</b> find a way to account for splits. 

<i>Initial idea:</i> So i think i can normalize by retroactively multiplying the dividend value by the split (eg  new_div = old_div * SPLIT) for each SPLIT record.  This would work for ATB-B.  will try later.  

<b>FALSE - need to adjust manually</b> Alternatively, sourcing dividend payouts from dividendhistory.org, the amounts are already adjusted but i have noticed some inconsistencies.  Going to try and account for splits.  

In [ ]:
for index, row in yahoo_split_check.iterrows():
    if not yahoo_split_check.empty:
        #print(row['Date'])
        test_div = new_div_yield.loc[new_div_yield['Date'] == row['Date']]
        print(test_div)
        print(row['value'])
        for index_2, row_2 in new_div_yield.iterrows():
            if row_2['Date']<=row['Date']:
                new_div_yield['value'] = new_div_yield['value']*row['value']
                #print("Old Div: {0}    New Div: {1}   Date: {2}  ".format(row_2['value'], row_2['Date']))
            #else:
                #print("no correction")
                
    else:
        print("No splits detected")
        


Once data is all adjusted and cleaned up - peak/trough finding/generation